## Define the Network

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [4]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [5]:
params = list(net.parameters())
print(params)
print(len(params))
print(params[0].size()) # conv1's weight

[Parameter containing:
tensor([[[[-0.0935, -0.1234, -0.1659, -0.1449,  0.1550],
          [-0.0472, -0.1853, -0.1347,  0.0652, -0.0466],
          [ 0.0337,  0.0468,  0.0887,  0.1281,  0.1893],
          [ 0.1251,  0.0382,  0.0152, -0.0964, -0.1670],
          [-0.0812, -0.1393,  0.1549,  0.1626,  0.1312]]],


        [[[-0.1990, -0.1763, -0.0378,  0.1835, -0.1818],
          [ 0.1413, -0.1463, -0.1652, -0.1004,  0.0222],
          [ 0.0883, -0.1975, -0.1320, -0.1711, -0.1534],
          [-0.1427, -0.0447, -0.0949, -0.0819,  0.0715],
          [ 0.1211,  0.1461,  0.0729,  0.1633,  0.1005]]],


        [[[-0.1582,  0.0401, -0.1298, -0.0755,  0.1476],
          [-0.0263, -0.0713,  0.0220,  0.0936,  0.0173],
          [ 0.0956,  0.1053,  0.0343, -0.1279, -0.0160],
          [ 0.1003,  0.0375,  0.0306, -0.1645,  0.0052],
          [ 0.1928,  0.1258, -0.1043, -0.0711,  0.1627]]],


        [[[ 0.0954,  0.0985, -0.1219, -0.1319,  0.1388],
          [ 0.0910,  0.1240,  0.0067,  0.0341, -0.099

In [8]:
input = torch.randn(1, 1, 32, 32)
print(input)
print(input.size())
out = net(input)
print(out)

tensor([[[[ 1.4275,  1.4027, -0.2897,  ..., -0.6229,  0.3778,  0.2805],
          [-1.3784,  1.1511,  1.7643,  ..., -0.3824,  0.5949, -0.4148],
          [ 0.4911,  0.8481,  1.4498,  ...,  0.8626,  0.9285, -0.8873],
          ...,
          [ 1.9511, -1.7569, -0.6933,  ..., -1.0575,  1.4084, -0.8783],
          [ 1.2747, -0.2258,  0.4504,  ...,  0.0764,  0.3686, -0.7566],
          [ 2.7049,  0.1896,  0.0358,  ...,  0.2842, -1.2235,  1.3002]]]])
torch.Size([1, 1, 32, 32])
tensor([[-0.0074,  0.0373,  0.0268,  0.0352, -0.0351,  0.0945,  0.0595, -0.0143,
         -0.0348,  0.1634]], grad_fn=<AddmmBackward>)


In [10]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [11]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.0719, grad_fn=<MseLossBackward>)


In [12]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [13]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0149,  0.0150,  0.0255, -0.0072,  0.0089, -0.0057])


In [14]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [15]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop
optimizer.zero_grad() # zero the gradient buffers. Because gradients are accumulated. what does this mean?
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # Does the update